# ETL

Importamos las librerias que vamos a ocupar

In [60]:
import pandas as pd
import re
import ast
from collections import Counter

Creamos el Dataframe importando el archivo "steam_games.json"

In [61]:
rows = []
with open('datasets/steam_games.json') as  f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)

Mostramos el dataframe para verificar que se haya cargado de la forma correcta.

In [62]:
df

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,NaN,NaN
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,NaN,NaN
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews,NaN


### Empiezan las transformaciones sobre la columna 'release_date'

Primero contamos cuantos valores nulos existen dentro del campo 'release_date'

In [63]:
null_count = df['release_date'].isnull().sum()

print("Número de valores nulos en la columna 'release_date':", null_count)

Número de valores nulos en la columna 'release_date': 2067


Cambiamos los valores Nulos por una cadena de texto vacia, esto para evitar errores durante el proceso de transformación.

In [64]:
df['release_date'] = df['release_date'].fillna('')

Creamos una lista de años esto para usarla en la extracción de los años del campo release_date ya que es la unica información que nos es relevante dentro de esta columna.

In [65]:
lista_anios = [str(anio) for anio in range(1970, 2023)]

Creamos un metodo para poder extraer unicamente los años de la de la columna release_date.

In [66]:
def extract_years(df, years_list, text_column, new_column):
    # Inicializar una lista para guardar el año encontrado para cada celda
    extracted_years = []

    # Recorrer cada celda del DataFrame en la columna de texto
    for text in df[text_column]:
        # Buscar cada año en la lista years_list en la celda de texto
        for year in years_list:
            if re.search(year, text):
                # Si se encuentra el año, agregarlo a la lista extracted_years
                extracted_years.append(year)
                break
        else:
            # Si no se encuentra ningún año, agregar una cadena vacía a extracted_years
            extracted_years.append('')

    # Agregar la lista extracted_years como una nueva columna en el DataFrame
    df[new_column] = extracted_years

    return df

# Llamar al método para extraer los años y guardarlos en la nueva columna
df = extract_years(df, lista_anios, 'release_date', 'anio_lanzamiento')

# Imprimir el DataFrame modificado
#print(df)

Verificamos que la nueva columna se haya creado correctamente.

In [67]:
df

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore,anio_lanzamiento
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN,2018
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,NaN,NaN,2018
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,NaN,NaN,2018
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews,NaN,2017


Contamos los numero de valores nulos que existen dentro de la columna 'genres'.

In [68]:
null_count = df['genres'].isnull().sum()

print("Número de valores nulos en la columna 'genres':", null_count)

Número de valores nulos en la columna 'genres': 3283


Cambiamos los valores nulos de la columna 'genres' por una cadena de texto vacio, esto para poder evitar errores durante la ejecución de las funciones.

In [69]:
df['genres'] = df['genres'].fillna('')

Contamos los valores nulos que existen dentro de la columna 'specs'.

In [70]:
null_count = df['specs'].isnull().sum()

print("Número de valores nulos en la columna 'specs':", null_count)

Número de valores nulos en la columna 'specs': 670


Cambiamos los valores nulos de la columna 'specs' por una cadena de texto vacio, esto para poder evitar errores durante la ejecución de las funciones.

In [71]:
df['specs'] = df['specs'].fillna('')

Contamos los valores nulos que existen dentro de la columna 'sentiment'.

In [72]:
null_count = df['sentiment'].isnull().sum()

print("Número de valores nulos en la columna 'sentiment':", null_count)

Número de valores nulos en la columna 'sentiment': 7182


Cambiamos los valores nulos de la columna 'sentiment' por una cadena de texto vacio, esto para poder evitar errores durante la ejecución de las funciones.

In [73]:
df['sentiment'] = df['sentiment'].fillna('')

Visualizamos los distintos valores que existen dentro de la columna 'sentiment' para poder crear una lista que usaremos en la función de análisis de sentimiento

In [74]:
df['sentiment'].value_counts()

sentiment
                           7182
Mixed                      4103
Very Positive              3868
Positive                   3281
Mostly Positive            2744
1 user reviews             2496
2 user reviews             1756
3 user reviews             1231
4 user reviews              964
5 user reviews              846
Mostly Negative             802
6 user reviews              756
7 user reviews              619
8 user reviews              537
9 user reviews              488
Overwhelmingly Positive     303
Negative                    123
Very Negative                29
Overwhelmingly Negative       7
Name: count, dtype: int64

Contamos los valores nulos que existen dentro de la columna 'metascore'.

In [75]:
null_count = df['metascore'].isnull().sum()

print("Número de valores nulos en la columna 'metascore':", null_count)

Número de valores nulos en la columna 'metascore': 29458


Contamos los valores con el valor 'NA' que existen dentro de la columna 'metascore'.

In [76]:
df['metascore'].value_counts()['NA']


70

Cambiamos los valores nulos de la columna 'metascore' por una cadena de texto vacio, esto para poder evitar errores durante la ejecución de las funciones.

In [77]:
df['metascore'] = df['metascore'].fillna('')

Cambiamos los valores 'NA' de la columna 'metascore' por una cadena de texto vacio, esto para poder evitar errores durante la ejecución de las funciones.

In [78]:
df['metascore'] = df['metascore'].replace('NA', '')

In [79]:
valor_buscar = 'free'
contador_de_palabras = df['price'].str.contains(valor_buscar, case=False).sum()

print("Número de registros que contienen la palabra 'free':", contador_de_palabras)

Número de registros que contienen la palabra 'free': 1900


In [80]:
null_count = df['price'].isnull().sum()

print("Número de valores nulos en la columna 'price':", null_count)

Número de valores nulos en la columna 'price': 1377


In [81]:
df['price'] = df['price'].fillna('')

In [ ]:
# Utilizando loc para buscar el valor y cambiar el valor en la celda específica
df.loc[df['price'].str.contains(valor_buscar, case=False), 'price'] = "0"

In [83]:
valor_buscar = ''
contador_de_palabras = df['genres'].str.contains(valor_buscar, case=False).sum()

print("Número de registros que contienen la palabra '':", contador_de_palabras)

Número de registros que contienen la palabra '': 3283


Creamos el archivo json nuevo

In [84]:
#df.to_csv('datasets/df_limpio.csv', index=False, encoding='utf-8')
df.to_json('datasets/df_limpio.json', orient='records', lines=True)